In [0]:
# 01_fetch_data.ipynb - Célula 1: Importações e Configuração da SparkSession

import requests
import json
import time
import os
import boto3
from botocore.exceptions import ClientError
from pyspark.sql import SparkSession

print("Bibliotecas importadas com sucesso.")

# A SparkSession já é automaticamente fornecida e configurada pelo Databricks.
# As credenciais AWS para S3 são gerenciadas pela IAM Role do CloudFormation.
spark = SparkSession.builder.appName("TFTDataFetcherDatabricksSimple").getOrCreate()

print("SparkSession configurada para acesso ao S3 via IAM Role.")

In [0]:
# 01_fetch_data.ipynb - Célula 2: Configuração da API Key (Hardcoded), Riot ID e Regiões

# !!! ATENÇÃO: COLOQUE SUA RIOT API KEY DIRETAMENTE AQUI !!!
# LEMBRE-SE DE REMOVÊ-LA ANTES DE SALVAR OU COMPARTILHAR O NOTEBOOK PUBLICAMENTE!
RIOT_API_KEY = "RGAPI-7edada16-cc12-4444-946a-2dccc51bd2d8" # Sua chave real

# Suas informações do Riot ID
GAME_NAME = "towwers" # Substitua pelo seu Nome de Jogo (a parte antes do #)
TAG_LINE = "tow" # Substitua pela sua Tagline (a parte depois do #, SEM o '#')

# Região de roteamento GLOBAL para Riot API (americas, europe, asia, sea)
GLOBAL_REGION = "americas" # Ajuste se você joga em outra região global

print("Configurações iniciais definidas.")

In [0]:
# 01_fetch_data.ipynb - Célula 3: Funções Auxiliares para Interagir com a Riot API

def get_puuid_from_riot_id(game_name: str, tag_line: str, api_key: str, global_region: str = "americas") -> str:
    url = f"https://{global_region}.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{game_name}/{tag_line}"
    headers = {"X-Riot-Token": api_key}
    response = requests.get(url, headers=headers)
    response.raise_for_status() # Levanta um erro para status de resposta HTTP ruins
    puuid = response.json().get("puuid")
    if not puuid:
        raise ValueError(f"PUUID não encontrado para {game_name}#{tag_line}. Verifique o Riot ID e a região.")
    return puuid

def get_tft_match_ids(puuid: str, api_key: str, count: int = 20, global_region: str = "americas") -> list[str]:
    url = f"https://{global_region}.api.riotgames.com/tft/match/v1/matches/by-puuid/{puuid}/ids"
    headers = {"X-Riot-Token": api_key}
    params = {"count": count}
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    return response.json()

def get_tft_match_details(match_id: str, api_key: str, global_region: str = "americas") -> dict:
    url = f"https://{global_region}.api.riotgames.com/tft/match/v1/matches/{match_id}"
    headers = {"X-Riot-Token": api_key}
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

print("Funções auxiliares para a API da Riot definidas.")

In [0]:
# 01_fetch_data.ipynb - Célula 4: Lógica Principal de Coleta de Dados

# 1. Obter o PUUID do seu Riot ID
print(f"Obtendo PUUID para {GAME_NAME}#{TAG_LINE}...")
try:
    my_puuid = get_puuid_from_riot_id(GAME_NAME, TAG_LINE, RIOT_API_KEY, GLOBAL_REGION)
    print(f"PUUID obtido: {my_puuid}")
except requests.exceptions.RequestException as e:
    print(f"ERRO ao obter PUUID: {e}. Verifique GAME_NAME, TAG_LINE, GLOBAL_REGION e Riot API Key.")
    my_puuid = None # Definir como None para evitar erros nas próximas etapas se o PUUID não for obtido

if my_puuid:
    # 2. Obter IDs das partidas de TFT
    num_matches_to_collect = 20 # Você pode ajustar este número (máx. 100 por requisição)
    print(f"\nObtendo os últimos {num_matches_to_collect} IDs de partidas de TFT...")
    try:
        match_ids = get_tft_match_ids(my_puuid, RIOT_API_KEY, num_matches_to_collect, GLOBAL_REGION)
        print(f"Total de IDs de partidas encontrados: {len(match_ids)}")
    except requests.exceptions.RequestException as e:
        print(f"ERRO ao obter IDs de partidas: {e}. Verifique o PUUID e a Riot API Key.")
        match_ids = [] # Definir como lista vazia se houver erro

    # 3. Obter detalhes completos de cada partida
    all_match_details = []
    errors_fetching_matches = []
    if match_ids:
        print("\nIniciando a coleta dos detalhes completos de cada partida...")
        for i, match_id in enumerate(match_ids):
            print(f"  Coletando detalhes da partida {i+1}/{len(match_ids)}: {match_id}...")
            try:
                details = get_tft_match_details(match_id, RIOT_API_KEY, GLOBAL_REGION)
                all_match_details.append(details)
                time.sleep(1.2) # Pausa para respeitar os limites de taxa da API
            except requests.exceptions.RequestException as e:
                print(f"  ERRO ao coletar detalhes da partida {match_id}: {e}")
                errors_fetching_matches.append(match_id)
                time.sleep(0.5) # Pausa mesmo em erro
            except json.JSONDecodeError as e:
                print(f"  ERRO de JSON ao coletar detalhes da partida {match_id}: {e}. Pulando esta.")
                errors_fetching_matches.append(match_id)

    print("\nColeta de detalhes das partidas finalizada.")
    print(f"Total de partidas detalhadas coletadas com sucesso: {len(all_match_details)}")
    if errors_fetching_matches:
        print(f"IDs das partidas que falharam na coleta: {errors_fetching_matches}")
    else:
        print("Todas as partidas foram coletadas com sucesso!")

    if all_match_details:
        print("\nEstrutura do JSON da primeira partida (chaves principais):")
        print(all_match_details[0].keys())
else:
    print("PUUID não obtido. Não foi possível prosseguir com a coleta de partidas.")

In [0]:
# 01_fetch_data.ipynb - Célula 5: Gravação dos Dados Brutos no AWS S3 e Criação da Tabela Externa

from pyspark.sql.functions import from_json, col, lit
from pyspark.sql.types import (
    StructType, StructField, StringType, ArrayType, LongType, IntegerType, DoubleType, BooleanType, MapType
)

# --- DEFINIÇÃO COMPLETA DO SCHEMA MANUALMENTE ---
# Baseado no JSON de exemplo da sua partida de TFT.

# Definindo o schema para as UNITS (Campeões no tabuleiro)
unit_schema = StructType([
    StructField("character_id", StringType(), True),
    StructField("itemNames", ArrayType(StringType()), True),
    StructField("name", StringType(), True),
    StructField("rarity", LongType(), True),
    StructField("tier", LongType(), True)
])

# Definindo o schema para as TRAITS (Sinergias)
trait_schema = StructType([
    StructField("name", StringType(), True),
    StructField("num_units", LongType(), True),
    StructField("style", LongType(), True),
    StructField("tier_current", LongType(), True),
    StructField("tier_total", LongType(), True)
])

# Definindo o schema para COMPANION (Pequeno Lendário)
companion_schema = StructType([
    StructField("content_ID", StringType(), True),
    StructField("item_ID", LongType(), True),
    StructField("skin_ID", LongType(), True),
    StructField("species", StringType(), True)
])

# Definindo o schema para MISSIONS (Missões, que é um mapa dinâmico)
# Usando StringType() para o valor para lidar com valores que podem não ser apenas números em todas as partidas
missions_schema = MapType(StringType(), StringType(), True)

# Definindo o schema para cada PARTICIPANT (Jogador na partida)
participant_schema = StructType([
    StructField("companion", companion_schema, True),
    StructField("gold_left", LongType(), True),
    StructField("last_round", LongType(), True),
    StructField("level", LongType(), True),
    StructField("missions", missions_schema, True),
    StructField("placement", LongType(), True),
    StructField("players_eliminated", LongType(), True),
    StructField("puuid", StringType(), True),
    StructField("riotIdGameName", StringType(), True),
    StructField("riotIdTagline", StringType(), True),
    StructField("time_eliminated", DoubleType(), True),
    StructField("total_damage_to_players", LongType(), True),
    StructField("traits", ArrayType(trait_schema), True),
    StructField("units", ArrayType(unit_schema), True),
    StructField("win", BooleanType(), True)
])

# Definindo o schema para INFO (Informações principais da partida)
info_schema = StructType([
    StructField("endOfGameResult", StringType(), True),
    StructField("gameCreation", LongType(), True), # Unix timestamp em milissegundos
    StructField("gameId", LongType(), True),
    StructField("game_datetime", LongType(), True), # Unix timestamp em milissegundos
    StructField("game_length", DoubleType(), True), # Duração em segundos (float)
    StructField("game_version", StringType(), True),
    StructField("mapId", LongType(), True),
    StructField("participants", ArrayType(participant_schema), True), # Array de schemas de participantes
    StructField("queueId", LongType(), True),
    StructField("queue_id", LongType(), True), # Inclua se existir, mesmo se for duplicado de queueId
    StructField("tft_game_type", StringType(), True),
    StructField("tft_set_core_name", StringType(), True),
    StructField("tft_set_number", LongType(), True)
])

# Definindo o schema para METADATA
metadata_schema = StructType([
    StructField("data_version", StringType(), True),
    StructField("match_id", StringType(), True),
    StructField("participants", ArrayType(StringType()), True) # Array de PUUIDs
])

# O SCHEMA COMPLETO DA PARTIDA
FULL_MATCH_SCHEMA = StructType([
    StructField("metadata", metadata_schema, True),
    StructField("info", info_schema, True)
])
print("Schema da partida de TFT definido.")

# --- Configurações do S3 e Unity Catalog ---
# SUBSTITUA PELO NOME REAL E EXATO DO SEU BUCKET S3
S3_BUCKET_NAME = "tft-data-pipeline-towwers"

# Caminho de destino no S3 para os dados brutos (formato JSON)
S3_RAW_OUTPUT_PATH = f"s3a://{S3_BUCKET_NAME}/raw/tft_matches_raw/"

# Nome completo da tabela no Unity Catalog (catalog.schema.table).
# Use o Catálogo e Schema que você criou: 'tft_analytics' e 'raw'.
UNITY_CATALOG_TABLE_NAME = "tft_analytics.raw.tft_matches_raw"

print(f"\nIniciando gravação de {len(all_match_details)} partidas no S3 em: {S3_RAW_OUTPUT_PATH}...")

if all_match_details:
    try:
        # 1. Cria um DataFrame Spark a partir da lista de strings JSON (compatível com Spark Connect)
        json_lines = [json.dumps(match_data, ensure_ascii=False) for match_data in all_match_details]
        df_json_strings = spark.createDataFrame([(line,) for line in json_lines], ["json_data"])
        
        # 2. Usa from_json para parsear a coluna de string JSON para uma estrutura de dados,
        # AGORA USANDO O SCHEMA MANUALMENTE DEFINIDO: FULL_MATCH_SCHEMA
        df_raw_spark = df_json_strings.withColumn(
            "parsed_data", from_json(col("json_data"), FULL_MATCH_SCHEMA)
        ).select("parsed_data.*") # Seleciona todos os campos da estrutura parseada

        # 3. Escreve o DataFrame no S3 no formato JSON.
        df_raw_spark.write.mode("overwrite").json(S3_RAW_OUTPUT_PATH)
        print(f"\nGravação dos arquivos JSON brutos no S3 concluída com sucesso em: {S3_RAW_OUTPUT_PATH}")

        # 4. Cria a Tabela Externa no Unity Catalog.
        print(f"Tentando criar tabela externa '{UNITY_CATALOG_TABLE_NAME}' apontando para '{S3_RAW_OUTPUT_PATH}'...")
        
        # Usamos saveAsTable com as opções de format e path
        df_raw_spark.write.format("json") \
                          .option("path", S3_RAW_OUTPUT_PATH) \
                          .mode("overwrite") \
                          .saveAsTable(UNITY_CATALOG_TABLE_NAME)
        
        print(f"Tabela externa '{UNITY_CATALOG_TABLE_NAME}' criada com sucesso no Unity Catalog!")
        
        # 5. Verificação: Exibir a tabela criada
        print("\nVerificação (query SQL na tabela do Unity Catalog):")
        spark.sql(f"SELECT COUNT(*) FROM {UNITY_CATALOG_TABLE_NAME}").show()
        spark.sql(f"SELECT * FROM {UNITY_CATALOG_TABLE_NAME} LIMIT 5").show(truncate=False)

    except Exception as e:
        print(f"\nERRO na gravação de dados ou criação da tabela: {e}")
        print("Verifique:")
        print(f"  - O nome do bucket S3 '{S3_BUCKET_NAME}' está correto e exato.")
        print(f"  - As permissões da IAM Role configurada pelo CloudFormation permitem escrita E leitura no bucket S3.")
        print(f"  - Se você tem as permissões 'CREATE TABLE' e 'USE SCHEMA' no esquema '{UNITY_CATALOG_TABLE_NAME.split('.')[1]}'.")
        print(f"  - Certifique-se de que o schema manual (FULL_MATCH_SCHEMA) está EXATO com a estrutura do seu JSON em todas as partidas.")
        print(f"  - Detalhes do erro: {e}")
else:
    print("Nenhum dado de partida coletado para gravar no S3.")